<a href="https://colab.research.google.com/github/c0pper/telegram-question-answering/blob/main/long_form_qa_it.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- https://www.youtube.com/watch?v=L8U-pm-vZ4c
- https://towardsdatascience.com/long-form-qa-beyond-eli5-an-updated-dataset-and-approach-319cb841aabb

In [ ]:
!pip install -qU datasets pinecone-client sentence-transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 KB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.3 MB/s eta 0:

In [ ]:
RETRIEVER_MODEL_NAME = "efederici/mmarco-sentence-BERTino"
TOKENIZER_MODEL_NAME = "efederici/bart-lfqa-it"
GENERATOR_MODEL_NAME = "efederici/bart-lfqa-it"
pinecone_index_name = "abstractive-question-answering-telegram-chat"

# Dataset loading

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = "1GUZqz3t7HYH_UmPebixnUYFhfYvD2FTw"
downloaded = drive.CreateFile({'id': file_id})

In [ ]:
import json

json = json.loads(downloaded.GetContentString())

In [ ]:
import pandas as pd

df = pd.DataFrame(json["messages"])
print(df.columns)
df = df[["text", "from", "from_id", "date"]]
df = df[df['from_id'] == 'user1748826398']
print(len(df))
df = df[df['text'].apply(type) == str]
print(len(df))

Index(['id', 'type', 'date', 'date_unixtime', 'actor', 'actor_id', 'action',
       'title', 'members', 'text', 'text_entities', 'from', 'from_id',
       'forwarded_from', 'file', 'thumbnail', 'media_type', 'sticker_emoji',
       'width', 'height', 'inviter', 'photo', 'reply_to_message_id',
       'mime_type', 'duration_seconds', 'place_name', 'address',
       'location_information', 'via_bot', 'edited', 'edited_unixtime',
       'performer', 'contact_information', 'live_location_period_seconds',
       'game_title', 'game_description', 'game_link', 'game_message_id',
       'score', 'poll', 'message_id', 'schedule_date', 'duration'],
      dtype='object')
18027
14990


In [ ]:
len_filter_df = df[df['text'].str.len() > 25]
print(len(len_filter_df))
len_filter_df.head(30)

5613


,text,from,from_id,date
96894,Ah grazie amici come prima,Lorenzo Valitutto,user1748826398,2021-05-01T17:22:29
96908,che mi hai regalato tu Giuppa,Lorenzo Valitutto,user1748826398,2021-05-01T17:26:36
96921,Ambrogio tu e Daniele vi somigliate parecchio,Lorenzo Valitutto,user1748826398,2021-05-01T17:32:02
96933,Estate 2020 quando potevo uscire,Lorenzo Valitutto,user1748826398,2021-05-01T17:37:55
96936,Ma io vi mando quelli che ho sul pc,Lorenzo Valitutto,user1748826398,2021-05-01T17:40:24
96939,Parole incomprensibili di Lorenzo Valitutto,Lorenzo Valitutto,user1748826398,2021-05-01T17:43:51
96941,Un ricordo del passato Giuppa!,Lorenzo Valitutto,user1748826398,2021-05-01T17:46:47
96966,La vita oltre la morte è da caricare,Lorenzo Valitutto,user1748826398,2021-05-01T17:59:33
96968,Le chiacchiere di carnevale,Lorenzo Valitutto,user1748826398,2021-05-01T18:00:02
96969,Video della Ford Fiction (Fusion),Lorenzo Valitutto,user1748826398,2021-05-01T18:01:12


In [ ]:
len_filter_df.sample(20)

,text,from,from_id,date
140655,IL LIBRO SEGRETO DI DENNY PETTINATI\n\n\nUn gi...,Lorenzo Valitutto,user1748826398,2023-03-23T11:36:52
105130,Innocente come una risata di un bambino,Lorenzo Valitutto,user1748826398,2021-07-03T12:19:08
137023,"Sicignano da vivere, ce l'ha ancora Federico L...",Lorenzo Valitutto,user1748826398,2023-02-18T13:13:10
106800,ma queste gif porno a me non mi piacciono,Lorenzo Valitutto,user1748826398,2021-07-22T12:00:36
115707,"Buona serata, scusatemi se vi siete arrabbiati.",Lorenzo Valitutto,user1748826398,2021-11-20T19:53:00
120610,"L'anno prossimo vado a ""Avanti un altro"" come ...",Lorenzo Valitutto,user1748826398,2022-09-20T11:30:30
140127,Ma è immediatamente intervenuta per la riparaz...,Lorenzo Valitutto,user1748826398,2023-03-16T14:56:21
127665,"Se avete già visto degli altri ingredienti, su...",Lorenzo Valitutto,user1748826398,2022-11-03T21:28:36
99754,"Buongiorno amici come prima, volevo dirvi che ...",Lorenzo Valitutto,user1748826398,2021-05-16T11:57:20
116416,L'altra volta siamo andati alla messa con il g...,Lorenzo Valitutto,user1748826398,2021-11-25T11:57:22


In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# Retriever setup

In [ ]:
from sentence_transformers import SentenceTransformer

# load retriever model from huggingface
retriever = SentenceTransformer(
    RETRIEVER_MODEL_NAME,
    device=device
)

# Pinecone setup

In [ ]:
import pinecone

pinecone.init(
    api_key=input("Pinecone api-key: "),
    environment="us-east4-gcp"
)

Pinecone api-key: 4d17ded1-9b15-4f43-a956-9c603149a073


In [ ]:
# check if index exists
if pinecone_index_name not in pinecone.list_indexes():
  pinecone.create_index(
      pinecone_index_name,
      dimension=retriever.get_sentence_embedding_dimension(),
      metric="cosine"
  )

  index = pinecone.Index(pinecone_index_name)
else:
  index = pinecone.Index(pinecone_index_name)

# Generating embedding

In [ ]:
from tqdm.auto import tqdm

# generating embedding
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
  # find end of batch
  i_end = min(i+batch_size, len(df))
  # extract batch
  batch = df.iloc[i:i_end]
  # generate embedding for batch
  emb = retriever.encode(batch["text"].tolist()).tolist()
  # get metadata
  meta = batch.to_dict(orient="records")
  # create unique ids
  ids = [f"{idx}" for idx in range(i, i_end)]
  # add all to upsert list
  to_upsert = list(zip(ids, emb, meta))
  # upsert/insert these records to pinecone
  _ = index.upsert(vectors=to_upsert)

# check we have all the vectors inside the index
index.describe_index_stats()

  0%|          | 0/235 [00:00<?, ?it/s]

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 14990}},
 'total_vector_count': 14990}

# Generator setup
BART model using transformers

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained(TOKENIZER_MODEL_NAME)
generator = BartForConditionalGeneration.from_pretrained(GENERATOR_MODEL_NAME)

In [ ]:
# helper functions to retrieve context passages from pinecone index and to format the query the way the generator expectes the input

def query_pinecone(query, top_k):
  # generate embeddings for the query
  xquery = retriever.encode([query]).tolist()
  # search pinecon index for contet passage with the answer
  xcontext = index.query(xquery, top_k=top_k, include_metadata=True)
  return xcontext

In [ ]:
def format_query(query, context):
  # extract passage_text from Pinecone search result and add the <P> tag
  # context = "<p> " + " <p> ".join([m['metadata']['text'] for m in context])
  context = [f"<p> {m['metadata']['text']}" for m in context]
  # concatenate all context passages
  context = " ".join(context)
  # concatenate the query and context passages
  query = f"Q: {query}\n\nC: {context}"
  # query = f"question: {query} context: {context}"
  return query

In [ ]:
# test query_pinecone()
query = "Chi è Giuseppe Perrotta?"
result = query_pinecone(query, top_k=5)
result

{'matches': [{'id': '4317',
              'metadata': {'date': datetime.datetime(2021, 8, 23, 12, 26, 42),
                           'from': 'Lorenzo Valitutto',
                           'from_id': 'user1748826398',
                           'text': 'Giuseppe Perrotta un collega poliziotto'},
              'score': 0.826561809,
              'values': []},
             {'id': '4571',
              'metadata': {'date': datetime.datetime(2021, 8, 31, 16, 20, 20),
                           'from': 'Lorenzo Valitutto',
                           'from_id': 'user1748826398',
                           'text': 'Giuseppe Perrotta: Corvonero'},
              'score': 0.786439836,
              'values': []},
             {'id': '14674',
              'metadata': {'date': datetime.datetime(2023, 3, 19, 7, 23, 45),
                           'from': 'Lorenzo Valitutto',
                           'from_id': 'user1748826398',
                           'text': 'Giuseppe Perrotta auguri'},
  

In [ ]:
from pprint import pprint

In [ ]:
# format the query in the way the generator expects the input
query = format_query(query, result["matches"])
pprint(query)

('Q: Chi è Giuseppe Perrotta?\n'
 '\n'
 'C: <p> Giuseppe Perrotta un collega poliziotto <p> Giuseppe Perrotta: '
 'Corvonero <p> Giuseppe Perrotta auguri <p> Che significa Giuseppe Perrotta '
 'removed Group Help? <p> Giuseppe Perrotta è amicissimo di tutti, ma nel suo '
 'cuore splende tutto. Amico vicino e lontano.\n'
 'Così gentile egli è amato,\n'
 'alle volte dispone\n'
 'specialmente di affinità e\n'
 'allora finalmente (fine).')


In [ ]:
# function to generate answers
def generate_answer(query, gen_min_length=10, gen_maxlength=40):
  # tokenize the query to get input ids
  inputs = tokenizer([query], max_length=1024, return_tensors="pt")
  # use generator to predict output ids
  ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=gen_min_length, max_length=gen_maxlength)
  # use tokenizer to decode the output ids
  answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)[0]
  return pprint(answer)

In [ ]:
generate_answer(query)

('Ciro Vitiello è il nome di un personaggio famoso in tutto il mondo.\n'
 '\n'
 'Ciro Vitiello è il nome di un personaggio famoso in tutto')


In [ ]:
query = "Dov'è Ciro Vitiello?"
context = query_pinecone(query, top_k=5) # give more infomation to generate answer from
query = format_query(query, context["matches"])
generate_answer(query)

('Ciro Vitiello è il nome di un personaggio famoso in tutto il mondo.\n'
 '\n'
 'Ciro Vitiello è il nome di un personaggio famoso in tutto')


# Explainability
To confirm the answer is correct we can check the contexts used to generate it

In [ ]:
for doc in context["matches"]:
  print(doc["metadata"]["text"], end="\n---\n")

Ieri Giuppa mi ha detto che il covid19 è finito
---
per covid19
---
Chiusa per Covid
---


In this case the answer looks correct. However if we ask a question and no relevant contexts are retrieved, the generator will typically return nonsensical or false answers, like with this question about covid-19

In [ ]:
query = "where did COVID-19 originate?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

'I content are intent an an an an an a'


In [ ]:
def ask(query, top_k=5, gen_minlength=10, gen_maxlength=40):
  context = query_pinecone(query, top_k=top_k)
  query = format_query(query, context["matches"])
  generate_answer(query)

  print("\n\nContext:\n")
  for doc in context["matches"]:
    print(doc["metadata"]["date"], doc["metadata"]["text"], end="\n---\n")

In [ ]:
ask("Quando c'è la sagra delle castagne a Sicignano?", top_k=10, gen_minlength=50, gen_maxlength=100)

('Ci vediamo direttamente alla sagra delle castagne a Sicignano <p> SIETE '
 'INVITATI TUTTI LA TERZA DOMENICA DI OTTOBRE ALLA SAGRA DELLE CAS')


Context:

2022-10-16 08:06:02 Ci vediamo direttamente alla sagra delle castagne a Sicignano
---
2022-09-09 11:41:00 SIETE INVITATI TUTTI LA TERZA DOMENICA DI OTTOBRE ALLA SAGRA DELLE CASTAGNE A SICIGNANO DEGLI ALBURNI
---
2021-09-25 10:23:41 la sagra delle castagne non si fa
---
2021-05-09 16:37:21 La sagra delle castagne, ci siete stati?
---
2021-10-13 12:29:08 Sentite una cosa, le persone il 17 ottobre vengono lo stesso a Sicignano anche se la sagra non si fa, voi potreste per me?
---
2022-10-23 10:02:16 Buongiorno a tutti coloro che oggi si sono svegliati con la fine della sagra delle castagne di Sicignano Degli Alburni.
---
2021-08-23 12:22:37 Ci sarete alla sagra delle castagne?
---
2021-07-31 12:48:05 realizzata il 18, 19 agosto a Sicignano
---
2021-05-17 12:13:40 alla sagra delle castagne se la fanno
---
2021-09-25 10:50:35 Delirio a 